# About this Notebook

Bayesian probabilistic matrix factorization (BPMF) is a classical model in the recommender system field. In the following, we will discuss:

- What the BPMF is?

- How to implement BPMF mainly using Python `Numpy` with high efficiency?

- How to make data imputations with real-world spatiotemporal datasets?

If you want to know more about BPMF, please read this article:

> Ruslan Salakhutdinov, Andriy Mnih, 2008. [**Bayesian probabilistic matrix factorization using Markov chain Monte Carlo**](https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf). Proceedings of the 25th International Conference on Machine Learning (*ICML 2008*), Helsinki, Finland. [[Matlab code (official)](https://www.cs.toronto.edu/~rsalakhu/BPMF.html)]

## Quick Run

This notebook is publicly available at [https://github.com/xinychen/tensor-learning](https://github.com/xinychen/tensor-learning).


In [1]:
import numpy as np
from numpy.linalg import inv as inv
from numpy.random import normal as normrnd
from scipy.linalg import khatri_rao as kr_prod
from scipy.stats import wishart
from numpy.linalg import solve as solve
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [3]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [4]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

# Part 2: Bayesian Probabilistic Matrix Factorization (BPMF)



In [5]:
def sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau, beta0 = 1, vargin = 0):
    """Sampling N-by-R factor matrix W and its hyperparameters (mu_w, Lambda_w)."""
    
    dim1, rank = W.shape
    W_bar = np.mean(W, axis = 0)
    temp = dim1 / (dim1 + beta0)
    var_mu_hyper = temp * W_bar
    var_W_hyper = inv(np.eye(rank) + cov_mat(W, W_bar) + temp * beta0 * np.outer(W_bar, W_bar))
    var_Lambda_hyper = wishart.rvs(df = dim1 + rank, scale = var_W_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim1 + beta0) * var_Lambda_hyper)
    
    if dim1 * rank ** 2 > 1e+8:
        vargin = 1
    
    if vargin == 0:
        var1 = X.T
        var2 = kr_prod(var1, var1)
        var3 = (var2 @ tau_ind.T).reshape([rank, rank, dim1]) + var_Lambda_hyper[:, :, np.newaxis]
        var4 = var1 @ tau_sparse_mat.T + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
        for i in range(dim1):
            W[i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    elif vargin == 1:
        for i in range(dim1):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            var_mu = tau * Xt.T @ sparse_mat[i, pos0[0]] + var_Lambda_hyper @ var_mu_hyper
            var_Lambda = tau * Xt.T @ Xt + var_Lambda_hyper
            W[i, :] = mvnrnd_pre(solve(var_Lambda, var_mu), var_Lambda)
    
    return W

In [6]:
def sample_factor_x(tau_sparse_mat, tau_ind, W, X, beta0 = 1):
    """Sampling T-by-R factor matrix X and its hyperparameters (mu_x, Lambda_x)."""
    
    dim2, rank = X.shape
    X_bar = np.mean(X, axis = 0)
    temp = dim2 / (dim2 + beta0)
    var_mu_hyper = temp * X_bar
    var_X_hyper = inv(np.eye(rank) + cov_mat(X, X_bar) + temp * beta0 * np.outer(X_bar, X_bar))
    var_Lambda_hyper = wishart.rvs(df = dim2 + rank, scale = var_X_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim2 + beta0) * var_Lambda_hyper)
    
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ tau_ind).reshape([rank, rank, dim2]) + var_Lambda_hyper[:, :, np.newaxis]
    var4 = var1 @ tau_sparse_mat + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
    for t in range(dim2):
        X[t, :] = mvnrnd_pre(solve(var3[:, :, t], var4[:, t]), var3[:, :, t])

    return X

In [7]:
def sample_precision_tau(sparse_mat, mat_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_mat - mat_hat) ** 2) * ind)
    return np.random.gamma(var_alpha, 1 / var_beta)

In [8]:
def BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter):
    """Bayesian Probabilistic Matrix Factorization, BPMF."""
    
    dim1, dim2 = sparse_mat.shape
    W = init["W"]
    X = init["X"]
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    ind = sparse_mat != 0
    pos_obs = np.where(ind)
    tau = 1
    W_plus = np.zeros((dim1, rank))
    X_plus = np.zeros((dim2, rank))
    temp_hat = np.zeros(sparse_mat.shape)
    show_iter = 200
    mat_hat_plus = np.zeros(sparse_mat.shape)
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau * ind
        tau_sparse_mat = tau * sparse_mat
        W = sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau)
        X = sample_factor_x(tau_sparse_mat, tau_ind, W, X)
        mat_hat = W @ X.T
        tau = sample_precision_tau(sparse_mat, mat_hat, ind)
        temp_hat += mat_hat
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_mat[pos_test], temp_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_mat[pos_test], temp_hat[pos_test])))
            temp_hat = np.zeros(sparse_mat.shape)
            print()
        if it + 1 > burn_iter:
            W_plus += W
            X_plus += X
            mat_hat_plus += mat_hat
    mat_hat = mat_hat_plus / gibbs_iter
    W = W_plus / gibbs_iter
    X = X_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_mat[pos_test], mat_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_mat[pos_test], mat_hat[pos_test])))
    print()
    
    return mat_hat, W, X

# Part 3: Data Organization

## 1) Matrix Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{f},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We express spatio-temporal dataset as a matrix $Y\in\mathbb{R}^{m\times f}$ with $m$ rows (e.g., locations) and $f$ columns (e.g., discrete time intervals),

$$Y=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{m1} & y_{m2} & \cdots & y_{mf} \\ \end{array} \right]\in\mathbb{R}^{m\times f}.$$

## 2) Tensor Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{nf},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We partition each time series into intervals of predifined length $f$. We express each partitioned time series as a matrix $Y_{i}$ with $n$ rows (e.g., days) and $f$ columns (e.g., discrete time intervals per day),

$$Y_{i}=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{n1} & y_{n2} & \cdots & y_{nf} \\ \end{array} \right]\in\mathbb{R}^{n\times f},i=1,2,...,m,$$

therefore, the resulting structure is a tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$.

# PeMS-4W

In [16]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 4 * 7), 0)
del data

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [17]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0486593
RMSE: 4.20742

Iter: 400
MAPE: 0.048604
RMSE: 4.20788

Iter: 600
MAPE: 0.0486043
RMSE: 4.20788

Iter: 800
MAPE: 0.0486043
RMSE: 4.20788

Iter: 1000
MAPE: 0.0486038
RMSE: 4.20788

Imputation MAPE: 0.0486039
Imputation RMSE: 4.20788

Running time: 100.51 minutes


In [18]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 4 * 7), 0)
del data

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [19]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.049038
RMSE: 4.22545

Iter: 400
MAPE: 0.0488331
RMSE: 4.22376

Iter: 600
MAPE: 0.0488337
RMSE: 4.22378

Iter: 800
MAPE: 0.0488334
RMSE: 4.22375

Iter: 1000
MAPE: 0.0488332
RMSE: 4.22377

Imputation MAPE: 0.0488333
Imputation RMSE: 4.22378

Running time: 102.76 minutes


In [20]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [21]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.051821
RMSE: 4.45542

Iter: 400
MAPE: 0.0517783
RMSE: 4.45911

Iter: 600
MAPE: 0.0517779
RMSE: 4.4591

Iter: 800
MAPE: 0.0517781
RMSE: 4.4591

Iter: 1000
MAPE: 0.0517777
RMSE: 4.45906

Imputation MAPE: 0.0517788
Imputation RMSE: 4.45914

Running time: 103.96 minutes


In [22]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [23]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0626843
RMSE: 5.50841

Iter: 400
MAPE: 0.0576987
RMSE: 5.06802

Iter: 600
MAPE: 0.05744
RMSE: 5.07271

Iter: 800
MAPE: 0.0576504
RMSE: 5.08866

Iter: 1000
MAPE: 0.0576505
RMSE: 5.08804

Imputation MAPE: 0.0576488
Imputation RMSE: 5.08817

Running time: 102.66 minutes


# PeMS-8W

In [24]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 8 * 7), 0)
del data

missing_rate = 0.3

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [25]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0515199
RMSE: 4.42065

Iter: 400
MAPE: 0.0515961
RMSE: 4.4274

Iter: 600
MAPE: 0.0515959
RMSE: 4.4274

Iter: 800
MAPE: 0.0515961
RMSE: 4.4274

Iter: 1000
MAPE: 0.0515959
RMSE: 4.4274

Imputation MAPE: 0.0515957
Imputation RMSE: 4.4274

Running time: 194.63 minutes


In [26]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
random_mat = ten2mat(np.random.rand(data.values.shape[0], 288, 8 * 7), 0)
del data

missing_rate = 0.7

### Random missing (RM) scenario:
sparse_mat = np.multiply(dense_mat, np.round(random_mat + 0.5 - missing_rate))

In [27]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0518987
RMSE: 4.43969

Iter: 400
MAPE: 0.0517679
RMSE: 4.43836

Iter: 600
MAPE: 0.0517675
RMSE: 4.43835

Iter: 800
MAPE: 0.0517679
RMSE: 4.43836

Iter: 1000
MAPE: 0.0517675
RMSE: 4.43835

Imputation MAPE: 0.0517675
Imputation RMSE: 4.43836

Running time: 192.17 minutes


In [28]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [29]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0532577
RMSE: 4.55766

Iter: 400
MAPE: 0.0532976
RMSE: 4.56359

Iter: 600
MAPE: 0.053297
RMSE: 4.56356

Iter: 800
MAPE: 0.0532974
RMSE: 4.56357

Iter: 1000
MAPE: 0.0532979
RMSE: 4.5636

Imputation MAPE: 0.0532974
Imputation RMSE: 4.56358

Running time: 190.65 minutes


In [30]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_mat = data.values
dense_tensor = mat2ten(dense_mat, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_mat = ten2mat(np.multiply(dense_tensor, binary_tensor), 0)
del dense_tensor, binary_tensor

In [31]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0550929
RMSE: 4.70189

Iter: 400
MAPE: 0.055128
RMSE: 4.71909

Iter: 600
MAPE: 0.0551271
RMSE: 4.71906

Iter: 800
MAPE: 0.0551268
RMSE: 4.71904

Iter: 1000
MAPE: 0.055128
RMSE: 4.71908

Imputation MAPE: 0.0551276
Imputation RMSE: 4.71916

Running time: 192.39 minutes


# London 1-M data

In [11]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = ten2mat(np.load('../datasets/London-data-set/hourly_speed_mat.npy'),0)
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

(219369, 720)


In [13]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0911408
RMSE: 2.206

Iter: 400
MAPE: 0.091145
RMSE: 2.20733

Iter: 600
MAPE: 0.0911443
RMSE: 2.20731

Iter: 800
MAPE: 0.0911441
RMSE: 2.20732

Iter: 1000
MAPE: 0.0911455
RMSE: 2.20733

Imputation MAPE: 0.0911463
Imputation RMSE: 2.2073

Running time: 56.78 minutes


In [14]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = ten2mat(np.load('../datasets/London-data-set/hourly_speed_mat.npy'),0)
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

In [15]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0939606
RMSE: 2.2713

Iter: 400
MAPE: 0.0940404
RMSE: 2.27407

Iter: 600
MAPE: 0.0940432
RMSE: 2.27408

Iter: 800
MAPE: 0.0940461
RMSE: 2.2741

Iter: 1000
MAPE: 0.0940405
RMSE: 2.27406

Imputation MAPE: 0.0940452
Imputation RMSE: 2.27407

Running time: 56.85 minutes


In [16]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = ten2mat(np.load('../datasets/London-data-set/hourly_speed_mat.npy'),0)
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

In [17]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0925648
RMSE: 2.24517

Iter: 400
MAPE: 0.0926499
RMSE: 2.24728

Iter: 600
MAPE: 0.0926493
RMSE: 2.24734

Iter: 800
MAPE: 0.0926427
RMSE: 2.24729

Iter: 1000
MAPE: 0.0926503
RMSE: 2.24735

Imputation MAPE: 0.0926499
Imputation RMSE: 2.24734

Running time: 53.95 minutes


In [18]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = ten2mat(np.load('../datasets/London-data-set/hourly_speed_mat.npy'),0)
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

In [19]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 20
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0996579
RMSE: 2.41299

Iter: 400
MAPE: 0.0998302
RMSE: 2.41553

Iter: 600
MAPE: 0.0999059
RMSE: 2.41747

Iter: 800
MAPE: 0.0998985
RMSE: 2.41746

Iter: 1000
MAPE: 0.0999051
RMSE: 2.41743

Imputation MAPE: 0.099904
Imputation RMSE: 2.41743

Running time: 54.12 minutes


# Guangzhou 2-M data

In [20]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.3

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

In [21]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 80
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0952559
RMSE: 4.04604

Iter: 400
MAPE: 0.095387
RMSE: 4.05625

Iter: 600
MAPE: 0.0954053
RMSE: 4.05689

Iter: 800
MAPE: 0.0953832
RMSE: 4.0564

Iter: 1000
MAPE: 0.09537
RMSE: 4.05596

Imputation MAPE: 0.0954006
Imputation RMSE: 4.05645

Running time: 75.16 minutes


In [22]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.7

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

In [23]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 80
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.105184
RMSE: 4.414

Iter: 400
MAPE: 0.10562
RMSE: 4.44005

Iter: 600
MAPE: 0.105606
RMSE: 4.43907

Iter: 800
MAPE: 0.1056
RMSE: 4.43941

Iter: 1000
MAPE: 0.105629
RMSE: 4.44034

Imputation MAPE: 0.105597
Imputation RMSE: 4.43939

Running time: 75.85 minutes


In [24]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')['random_matrix']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.3

## Non-random missing (NM)
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
sparse_mat = np.multiply(dense_mat, binary_mat)

In [25]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.102993
RMSE: 4.33674

Iter: 400
MAPE: 0.103039
RMSE: 4.34403

Iter: 600
MAPE: 0.103041
RMSE: 4.34394

Iter: 800
MAPE: 0.103038
RMSE: 4.34421

Iter: 1000
MAPE: 0.103048
RMSE: 4.34422

Imputation MAPE: 0.103043
Imputation RMSE: 4.34438

Running time: 7.81 minutes


In [26]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')['random_matrix']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.7

## Non-random missing (NM)
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
sparse_mat = np.multiply(dense_mat, binary_mat)

In [27]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
init = {"W": 0.1 * np.random.randn(dim1, rank), 
        "X": 0.1 * np.random.randn(dim2, rank)}
burn_iter = 1000
gibbs_iter = 200
BPMF(dense_mat, sparse_mat, init, rank, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.112633
RMSE: 4.70933

Iter: 400
MAPE: 0.112978
RMSE: 4.73468

Iter: 600
MAPE: 0.113305
RMSE: 4.75515

Iter: 800
MAPE: 0.113285
RMSE: 4.75474

Iter: 1000
MAPE: 0.113298
RMSE: 4.75618

Imputation MAPE: 0.113257
Imputation RMSE: 4.75376

Running time: 7.78 minutes


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>